### Importing Required Libraries

In [1]:
import requests
from dotenv import load_dotenv
import os


### Loading Environment Variables

In [30]:
# Load environment variables from .env file
load_dotenv()

# Retrieve the API key from environment variables
api_key = os.getenv("HUGGINGFACE_API_KEY")

if not api_key:
    print("Error: Missing API Key!")
    exit(1)


### Defining the API URL and Headers

In [31]:
# Define the API URL and headers with your Hugging Face API key
api_url = "https://api-inference.huggingface.co/models/distilbert-base-uncased-finetuned-sst-2-english"
headers = {
    "Authorization": f"Bearer {api_key}"
}


### Loading Negative Areas from File

In [32]:
# Load negative areas from a file
def load_negative_areas(file_path):
    try:
        with open(file_path, 'r') as file:
            return [line.strip() for line in file.readlines()]
    except FileNotFoundError:
        print(f"Error: {file_path} not found!")
        return []



### Function to Process Feedback

In [33]:
# Function to process feedback
def process_feedback(feedback):
    # Load the negative areas from the file
    negative_areas = load_negative_areas('negative_areas.txt')  # Ensure the file is in the same directory

    if not feedback.strip():
        print("Please enter some valid feedback!")
        return

    # Create the input payload for the API request
    data = {"inputs": feedback}

    # Make a POST request to Hugging Face's API
    response = requests.post(api_url, headers=headers, json=data)

    # Check if the response is successful and process the result
    if response.status_code == 200:
        result = response.json()  # If successful, process the response data
        print("\nAPI Response:", result)  # Debugging step to inspect the actual response structure
        
        # Extract the sentiment label
        try:
            sentiment = result[0][0]['label']  # Adjusted indexing
            print(f"\nSentiment Analysis Result:\n- Sentiment: {sentiment}")
            
            if sentiment == "NEGATIVE":
                print("  Negative feedback related to:")
                
                # Compare feedback with the negative areas
                found_areas = [area for area in negative_areas if area.lower() in feedback.lower()]
                
                if found_areas:
                    for area in found_areas:
                        print(f"  - {area}")
                else:
                    print("  No specific areas of negative feedback identified.")
        except (IndexError, KeyError) as e:
            print("Unexpected API response structure:", e)
    else:
        print(f"Error: {response.status_code}, {response.text}")


### Main Function to Get User Input and Call Processing Function

In [34]:
def main():
    # Input: Dynamic user feedback
    feedback = input("Please enter your feedback: ")
    
    # Call the function to process feedback
    process_feedback(feedback)


In [27]:
main()

Please enter your feedback:  My stay at this hotel was delightful. The check-in process was smooth, and the reception staff greeted me with warmth and professionalism. The room was spotless, with comfortable bedding and excellent air conditioning. The dining experience was exceptional, with delicious food and attentive staff. I appreciated the fast Wi-Fi, spacious parking facilities, and well-maintained pool and spa services. Overall, the staff went above and beyond to ensure a pleasant stay, and I would highly recommend this hotel to others.



API Response: [[{'label': 'POSITIVE', 'score': 0.9998680353164673}, {'label': 'NEGATIVE', 'score': 0.0001319767616223544}]]
Sentiment: POSITIVE


In [35]:
main()

Please enter your feedback:  The check-in process was chaotic, taking over an hour because the reception staff seemed poorly trained and disorganized. When I finally got to my room, I found that the air conditioning was not working, the room smelled musty, and the bathroom had stains on the sink. To make matters worse, the noise from neighboring rooms was unbearable, and despite complaining multiple times to the staff, nothing was done to address the issue. The lack of proper parking facilities added to the frustration as I had to park far from the hotel. Overall, this was one of the worst experiences I've had in a hotel.



API Response: [[{'label': 'NEGATIVE', 'score': 0.9997958540916443}, {'label': 'POSITIVE', 'score': 0.00020419068459887058}]]

Sentiment Analysis Result:
- Sentiment: NEGATIVE
  Negative feedback related to:
  - reception
  - facilities
  - check-in process
  - parking facilities


In [10]:
main()

Please enter your feedback:  I had high hopes for this hotel, but it failed miserably on all fronts. The dining area was unclean, with tables not cleared even after guests had left. Room service was incredibly slow; I had to wait over an hour for a simple order. The elevator was malfunctioning, and I had to climb several flights of stairs to reach my room. Adding to the woes, the laundry service returned my clothes late and with stains. The billing process was confusing and overcharged me for amenities I didn’t use. The overall lack of professionalism and attention to detail was appalling.



API Response: [[{'label': 'NEGATIVE', 'score': 0.9997768998146057}, {'label': 'POSITIVE', 'score': 0.0002231094113085419}]]
Sentiment: NEGATIVE
Analyzing areas that caused negative feedback...

Negative feedback related to:
- overcharged
- dining
- room service
- amenities


In [14]:
main()

Please enter your feedback:  My experience at this hotel was disappointing from start to finish. The valet service was slow and unprofessional, and the check-in process took forever. The room had blood stains on the pillow, and the bathroom was filthy. When I called for room service, I was put on hold for over 15 minutes, only to be told that they couldn't accommodate my request. The Wi-Fi was non-existent, and the noise from the hallway made it impossible to rest. The management seemed indifferent to the numerous issues I raised.



API Response: [[{'label': 'NEGATIVE', 'score': 0.9997290968894958}, {'label': 'POSITIVE', 'score': 0.00027091664378531277}]]
Sentiment: NEGATIVE
Analyzing areas that caused negative feedback...

Negative feedback related to:
- room service
- check-in process


In [16]:
main()

Please enter your feedback:  I chose this hotel based on its advertised amenities, but it was a complete letdown. The dining experience was terrible; the food was cold, and the staff seemed inattentive. The air conditioning in my room didn’t work properly, and the parking facilities were a nightmare, with no security in place. On top of that, the check-out process was delayed because the system was down, and no one offered an apology or solution. The overall attitude of the staff and the state of the facilities made this a regretful choice.



API Response: [[{'label': 'NEGATIVE', 'score': 0.9998032450675964}, {'label': 'POSITIVE', 'score': 0.0001967947173397988}]]
Sentiment: NEGATIVE
Analyzing areas that caused negative feedback...

Negative feedback related to:
- dining
- facilities
- amenities
- check-out process
- parking facilities
